<a href="https://colab.research.google.com/github/Matheus0697/LoginResponsive/blob/main/SistemaRecomendacaoPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# IDs das planilhas Google
sheet_id_filmes = '1QnKjOTBYDwItVJJiwlz46tBs5xR2EQyDKa6XJY9iD4o'
sheet_id_avaliacoes = '1ZOJCtiD1SAN9bLgVMgLSWPMqG6EoX5YCVWsLhD3SGow'

# URLs para acessar os dados das planilhas Google
url_filmes = f'https://docs.google.com/spreadsheets/d/{sheet_id_filmes}/export?format=csv'
url_avaliacoes = f'https://docs.google.com/spreadsheets/d/{sheet_id_avaliacoes}/export?format=csv'

# Carregando os dados
filmes = pd.read_csv(url_filmes)
avaliacoes = pd.read_csv(url_avaliacoes)

# Filtrando e renomeando colunas
filmes = filmes[['id', 'original_title', 'original_language', 'vote_count']]
filmes.columns = ['ID_FILME', 'TITULO', 'LINGUAGEM', 'QT_AVALIACOES']

avaliacoes = avaliacoes[['userId', 'movieId', 'rating']]
avaliacoes.columns = ['ID_USUARIO', 'ID_FILME', 'AVALIACAO']

# Removendo valores nulos
filmes.dropna(inplace=True)

# Selecionando usuários com mais de 999 avaliações
usuarios_ativos = avaliacoes['ID_USUARIO'].value_counts() > 999
usuarios_ativos = usuarios_ativos[usuarios_ativos].index
avaliacoes = avaliacoes[avaliacoes['ID_USUARIO'].isin(usuarios_ativos)]

# Filtrando filmes com mais de 999 avaliações e em inglês
filmes = filmes[(filmes['QT_AVALIACOES'] > 999) & (filmes['LINGUAGEM'] == 'en')]

# Convertendo ID_FILME para inteiro
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)

# Unindo DataFrames
avaliacoes_e_filmes = avaliacoes.merge(filmes, on='ID_FILME')

# Removendo duplicatas
avaliacoes_e_filmes.drop_duplicates(['ID_USUARIO', 'ID_FILME'], inplace=True)
avaliacoes_e_filmes.drop(columns='ID_FILME', inplace=True)

# Transformando em tabela pivô e preenchendo nulos com zero
filmes_pivot = avaliacoes_e_filmes.pivot_table(columns='ID_USUARIO', index='TITULO', values='AVALIACAO').fillna(0)

# Criando matriz sparsa
filmes_sparse = csr_matrix(filmes_pivot)

# Treinando o modelo KNN
modelo = NearestNeighbors(algorithm='brute')
modelo.fit(filmes_sparse)

# Listando os 10 filmes mais bem avaliados
filmes_media_avaliacoes = avaliacoes_e_filmes.groupby('TITULO')['AVALIACAO'].mean()
top_10_filmes = filmes_media_avaliacoes.sort_values(ascending=False).head(10)
print("Top 10 filmes mais bem avaliados:")
print(top_10_filmes)

# Input para o usuário digitar o nome do filme
filme_input = input("Digite o nome do filme para obter recomendações: ").strip().lower()

# Convertendo os títulos dos filmes para minúsculas
filmes_pivot_lower = filmes_pivot.copy()
filmes_pivot_lower.index = filmes_pivot_lower.index.str.lower()

# Fazendo recomendações
if filme_input in filmes_pivot_lower.index:
    distances, suggestions = modelo.kneighbors(filmes_pivot_lower.loc[[filme_input]].values)
    print(f"Recomendações para '{filme_input}':")
    for i in range(len(suggestions[0])):
        print(filmes_pivot.index[suggestions[0][i]])
else:
    print("Filme não encontrado. Por favor, verifique o nome e tente novamente.")


<ipython-input-20-0d3b7d8ab29d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes.dropna(inplace=True)


Top 10 filmes mais bem avaliados:
TITULO
Zodiac                                           5.000000
The Mummy Returns                                5.000000
Hulk                                             5.000000
The Green Mile                                   4.500000
Lawless                                          4.500000
Rush Hour                                        4.500000
The Lord of the Rings: The Return of the King    4.500000
The Bourne Supremacy                             4.500000
Dawn of the Dead                                 4.333333
Notting Hill                                     4.250000
Name: AVALIACAO, dtype: float64
Digite o nome do filme para obter recomendações: 300
Recomendações para '300':
300
Back to the Future
The Lovely Bones
Armageddon
Romeo + Juliet
